# Creating Ground Truth Dataset

In [349]:
import pandas as pd 
import json
import ast
import re

In [350]:
# Import sample data
sample_df = pd.read_parquet('project_b_samples_2k.parquet')
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""shipping_center"",""alternate"":[""fre...",0.996261,"[""http://www.goinpostaljacksonville.com/""]","[""https://www.facebook.com/463273470392736""]",None,"[""+19049989600""]",...,"[{""dataset"":""Microsoft"",""record_id"":""168884986...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""vehicle_shipping"",""alternate"":[""co...",0.77,"[""https://www.goinpostaljacksonville.com/""]",None,None,"[""9049989600""]","{""names"":{}}","[{""freeform"":""7643 Gate Pkwy Ste 104"",""localit..."
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Valley Transmission""}","{""primary"":""automotive_repair"",""alternate"":[""a...",0.996261,"[""http://valleytransca.com/""]","[""https://www.facebook.com/107150256087765""]",None,"[""+16194474353""]",...,"[{""dataset"":""Microsoft"",""record_id"":""112589990...","{""primary"":""Valley Transmission""}","{""primary"":""transmission_repair"",""alternate"":[...",0.77,"[""http://valleytransca.com/""]",None,None,"[""6194474353""]","{""names"":{}}","[{""freeform"":""1158 N 2nd St"",""locality"":""El Ca..."
2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Mazda Nelspruit""}","{""primary"":""car_dealer"",""alternate"":[""automoti...",0.997899,"[""http://bit.ly/NelspruitMazda""]","[""https://www.facebook.com/113070635372645""]",None,"[""+27137573800""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424934...","{""primary"":""Mazda Nelspruit""}","{""primary"":""used_car_dealer"",""alternate"":[""aut...",0.77,"[""https://nelspruitmazda.co.za/""]",None,None,"[""0137573800""]","{""names"":{}}","[{""freeform"":""49 Emnotweni Avenue"",""locality"":..."
3,1407374885933937,844424932146366,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Red Wing - Roswell, GA""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.770000,"[""https://stores.redwingshoes.com/roswell-ga""]",None,None,"[""7705584400""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424932...","{""primary"":""Red Wing""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.77,"[""http://stores.redwing.com/roswell-ga""]",None,None,"[""7705584400""]","{""names"":{}}","[{""freeform"":""875 Mansell Rd Ste B1"",""locality..."
4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Norauto España""}","{""primary"":""automotive_repair"",""alternate"":[""c...",0.996261,"[""https://centro.norauto.es/alacant/finestrat/...","[""https://www.facebook.com/295967644448975""]",None,"[""+34966803254""]",...,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Norauto""}","{""primary"":""brake_service_and_repair"",""alterna...",0.77,"[""http://centro.norauto.es/centre/alicante-fin...",None,None,"[""966803254""]","{""names"":{}}","[{""freeform"":""Calle Rei Jaume I 9"",""locality"":..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""lawyer"",""alternate"":[""criminal_def...",0.286713,"[""http://www.petraccimarin.it""]","[""https://www.facebook.com/121776774503515""]",None,"[""+39040660022""]",...,"[{""property"":"""",""dataset"":""FourSquare"",""record...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""Business 

In [351]:
# Import Yelp dataset (JSON Lines format)
yelp_df = pd.read_json('Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json', lines=True)
yelp_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [352]:
# Drop unneeded Yelp Columns
unimportant = ["latitude", "longitude", "stars", "review_count", "is_open", "attributes", "hours"]
yelp_df.drop(columns=unimportant, inplace=True)

In [353]:
# Assigns data types to attributes
def parse_category(x):
    if isinstance(x, (list, dict)) or pd.isna(x): return x
    try: return ast.literal_eval(x)
    except: return None

for c in ["sources", "names", "addresses", "categories", "websites", 
          "brand", "emails", "socials", "base_names","base_addresses",
          "base_categories", "base_websites", "base_brand", "base_emails", "base_socials"]:
    if c in sample_df: sample_df[c] = sample_df[c].apply(parse_category)

In [354]:
sample_df.map(type)

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'list'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'dict'>,<class 'list'>
1,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'list'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'dict'>,<class 'list'>
2,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'list'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'dict'>,<class 'list'>
3,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'dict'>,<class 'list'>
4,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'list'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'dict'>,<class 'list'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'list'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'dict'>,<class 'list'>
1996,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'list'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'dict'>,<class 'list'>
1997,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'NoneType'>,<class 'list'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'NoneType'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'dict'>,<class 'list'>
1998,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'list'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'list'>,<class 'str'>,<class 'dict'>,<class 'list'>


In [355]:
# Extract country information from address dictionaries
def extract_country(addr_list):
    """Extract country from address list (non-base attributes)"""
    if isinstance(addr_list, list) and addr_list:
        country = addr_list[0].get("country")
        if country:
            return country.strip().upper()
    return None

def extract_base_country(base_addr_list):
    """Extract country from base address list (base_ attributes)"""
    if isinstance(base_addr_list, list) and base_addr_list:
        country = base_addr_list[0].get("country")
        if country:
            return country.strip().upper()
    return None

# Create country columns
sample_df["country"] = sample_df["addresses"].apply(extract_country)
sample_df["base_country"] = sample_df["base_addresses"].apply(extract_base_country)

# Track conflicting countries between base and non-base
conflicting_countries = ((sample_df["country"] != sample_df["base_country"]) & 
                        (sample_df["country"].notna()) & 
                        (sample_df["base_country"].notna())).sum()

print(f"Number of rows with conflicting countries: {conflicting_countries}")
print(f"Total rows with both countries present: {((sample_df['country'].notna()) & (sample_df['base_country'].notna())).sum()}")
print(f"Percentage of conflicts: {conflicting_countries / len(sample_df) * 100:.2f}%")

# Display country distribution
print("\nCountry distribution (non-base):")
print(sample_df["country"].value_counts())
print("\nBase country distribution:")
print(sample_df["base_country"].value_counts())

# Compare countries between sources
print(f"\nCountry comparison:")
country_comparison = sample_df[["country", "base_country"]].value_counts()
print(country_comparison)

sample_df[["country", "base_country"]].head(10)

Number of rows with conflicting countries: 1
Total rows with both countries present: 2000
Percentage of conflicts: 0.05%

Country distribution (non-base):
country
US    960
DE    151
IT    110
BR     95
GB     94
JP     90
MX     75
TH     73
FR     70
CA     51
IN     44
AU     27
ES     22
AT     18
NL     10
MY     10
TR      9
ID      8
DK      7
CH      7
PL      7
SE      6
CZ      4
BE      4
PT      4
PH      3
AR      3
NZ      3
HU      3
FI      3
ZA      3
NO      2
PR      2
RU      2
IE      2
CO      2
OM      2
SG      1
CL      1
DO      1
BO      1
SK      1
GR      1
PE      1
LV      1
IQ      1
KR      1
CN      1
VN      1
RS      1
SR      1
Name: count, dtype: int64

Base country distribution:
base_country
US    961
DE    151
IT    110
BR     95
GB     94
JP     90
MX     75
TH     73
FR     70
CA     51
IN     44
AU     27
ES     22
AT     18
NL     10
MY     10
TR      9
ID      8
DK      7
CH      7
PL      7
SE      6
CZ      4
BE      4
PT      4
PH      3


,country,base_country
0,US,US
1,US,US
2,ZA,ZA
3,US,US
4,ES,ES
5,US,US
6,US,US
7,NL,NL
8,US,US
9,ES,ES


In [356]:
# Show rows with conflicting countries
conflicting_mask = ((sample_df["country"] != sample_df["base_country"]) & 
                   (sample_df["country"].notna()) & 
                   (sample_df["base_country"].notna()))

conflicting_rows = sample_df[conflicting_mask]

print(f"Rows with conflicting countries ({len(conflicting_rows)} total):")
print("=" * 60)

if len(conflicting_rows) > 0:
    # Show key columns for conflicting rows
    display_cols = ["id", "country", "base_country", "names", "base_names"]
    available_cols = [col for col in display_cols if col in sample_df.columns]
    
    print(conflicting_rows[available_cols])
    
    print("\nDetailed view of conflicts:")
    for i, (idx, row) in enumerate(conflicting_rows.head(3).iterrows()):
        print(f"\nConflict {i+1} (Row {idx}):")
        print(f"  Non-base country: {row['country']}")
        print(f"  Base country: {row['base_country']}")
        if 'names' in row and row['names']:
            print(f"  Non-base name: {row['names']}")
        if 'base_names' in row and row['base_names']:
            print(f"  Base name: {row['base_names']}")
        if 'addresses' in row and row['addresses']:
            print(f"  Non-base address: {row['addresses']}")
        if 'base_addresses' in row and row['base_addresses']:
            print(f"  Base address: {row['base_addresses']}")
else:
    print("No conflicting countries found!")

Rows with conflicting countries (1 total):
                                   id country base_country  \
932  08f4cee794ada052038e889f077842df      PR           US   

                     names                          base_names  
932  {'primary': 'PenFed'}  {'primary': 'PenFed Credit Union'}  

Detailed view of conflicts:

Conflict 1 (Row 932):
  Non-base country: PR
  Base country: US
  Non-base name: {'primary': 'PenFed'}
  Base name: {'primary': 'PenFed Credit Union'}
  Non-base address: [{'freeform': '7 Calle Tabonuco', 'locality': 'Guaynabo', 'postcode': '00968', 'country': 'PR'}]
  Base address: [{'freeform': 'Calle Tabonuco', 'locality': 'Guaynabo', 'region': 'PR', 'country': 'US', 'postcode': '00968'}]


In [357]:
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses,country,base_country
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'shipping_center', 'alternate': ['...",0.996261,[http://www.goinpostaljacksonville.com/],[https://www.facebook.com/463273470392736],None,"[""+19049989600""]",...,"{'primary': 'vehicle_shipping', 'alternate': [...",0.77,[https://www.goinpostaljacksonville.com/],None,None,"[""9049989600""]",{'names': {}},"[{'freeform': '7643 Gate Pkwy Ste 104', 'local...",US,US
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[http://valleytransca.com/],[https://www.facebook.com/107150256087765],None,"[""+16194474353""]",...,"{'primary': 'transmission_repair', 'alternate'...",0.77,[http://valleytransca.com/],None,None,"[""6194474353""]",{'names': {}},"[{'freeform': '1158 N 2nd St', 'locality': 'El...",US,US
2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Mazda Nelspruit'},"{'primary': 'car_dealer', 'alternate': ['autom...",0.997899,[http://bit.ly/NelspruitMazda],[https://www.facebook.com/113070635372645],None,"[""+27137573800""]",...,"{'primary': 'used_car_dealer', 'alternate': ['...",0.77,[https://nelspruitmazda.co.za/],None,None,"[""0137573800""]",{'names': {}},"[{'freeform': '49 Emnotweni Avenue', 'locality...",ZA,ZA
3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,[https://stores.redwingshoes.com/roswell-ga],None,None,"[""7705584400""]",...,"{'primary': 'shoe_store', 'alternate': ['onlin...",0.77,[http://stores.redwing.com/roswell-ga],None,None,"[""7705584400""]",{'names': {}},"[{'freeform': '875 Mansell Rd Ste B1', 'locali...",US,US
4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Norauto España'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[https://centro.norauto.es/alacant/finestrat/c...,[https://www.facebook.com/295967644448975],None,"[""+34966803254""]",...,"{'primary': 'brake_service_and_repair', 'alter...",0.77,[http://centro.norauto.es/centre/alicante-fine...,None,None,"[""966803254""]",{'names': {}},"[{'freeform': 'Calle Rei Jaume I 9', 'locality...",ES,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Studio Legale Associato Petracci ...,"{'primary': 'lawyer', 'alternate': ['criminal_...",0.286713,[http://www.petraccimarin.it],[https://www.facebook.com/121776774503515],None,"[""+39040660022""]",...,{'primary': 'Business and Professional Service...,1.00,[http://www.petraccimarin.it],None,None,"[""040 660022""]","{'wikidata': '', 'names': {'primary': 'Studio ...","[{'freeform': 'Via beccaria 3', 'locality': 'T...",IT,IT
1996,08fa80b035674b6a03379ca9c1ecea9f,50364256e4b0573431043dd4,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Farmácias Pague Menos'},"{'primary': 'pharmacy', 'alternate': ['pharmac...",0.792793,[https://www.paguemenos.com.br/],[https://www.facebook.com/2204406756483378],None,"[""+556740028282""]",...,"{'primary': 'Retail > Pharmacy', 'alternate': []}",1.00,[https://www.paguemenos.com.br],None,None,"[""(67) 3521-4016""]","{'wikidata': '', 'names': {'primary': 'Farmaci...","[{'freeform': 'Av. Antônio Trajano, 583', 'loc...",BR,BR
1997,08f5f1c34bc3419903e2d20614543143,4ccc446a566aa0931a5220fd,"[{'property': '', 'dataset': 'meta', 'record_

In [358]:
# Filter to keep only rows where BOTH country and base_country are "US"
print(f"Original dataset size: {len(sample_df)} rows")

# Create filter condition
us_filter = (sample_df["country"] == "US") & (sample_df["base_country"] == "US")

# Apply filter
sample_df_us = sample_df[us_filter].copy()

print(f"Filtered dataset size: {len(sample_df_us)} rows")
print(f"Removed {len(sample_df) - len(sample_df_us)} rows")
print(f"Kept {len(sample_df_us) / len(sample_df) * 100:.2f}% of original data")

# Verify the filter worked
print(f"\nCountry distribution in filtered data:")
print(f"Country values: {sample_df_us['country'].value_counts()}")
print(f"Base country values: {sample_df_us['base_country'].value_counts()}")

# Update sample_df to the filtered version
sample_df = sample_df_us
print(f"\nUpdated sample_df to US-only data: {len(sample_df)} rows")
sample_df.head()

Original dataset size: 2000 rows
Filtered dataset size: 960 rows
Removed 1040 rows
Kept 48.00% of original data

Country distribution in filtered data:
Country values: country
US    960
Name: count, dtype: int64
Base country values: base_country
US    960
Name: count, dtype: int64

Updated sample_df to US-only data: 960 rows


,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses,country,base_country
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'shipping_center', 'alternate': ['...",0.996261,[http://www.goinpostaljacksonville.com/],[https://www.facebook.com/463273470392736],None,"[""+19049989600""]",...,"{'primary': 'vehicle_shipping', 'alternate': [...",0.77,[https://www.goinpostaljacksonville.com/],None,None,"[""9049989600""]",{'names': {}},"[{'freeform': '7643 Gate Pkwy Ste 104', 'local...",US,US
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[http://valleytransca.com/],[https://www.facebook.com/107150256087765],None,"[""+16194474353""]",...,"{'primary': 'transmission_repair', 'alternate'...",0.77,[http://valleytransca.com/],None,None,"[""6194474353""]",{'names': {}},"[{'freeform': '1158 N 2nd St', 'locality': 'El...",US,US
3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,[https://stores.redwingshoes.com/roswell-ga],None,None,"[""7705584400""]",...,"{'primary': 'shoe_store', 'alternate': ['onlin...",0.77,[http://stores.redwing.com/roswell-ga],None,None,"[""7705584400""]",{'names': {}},"[{'freeform': '875 Mansell Rd Ste B1', 'locali...",US,US
5,1125899912143279,1407374890190043,"[{'dataset': 'Microsoft', 'record_id': '112589...",{'primary': 'Newport Institute'},"{'primary': 'doctor', 'alternate': ['health_an...",0.770000,[http://www.newportinstitute.com/?utm_source=s...,None,None,"[""8445450602""]",...,"{'primary': 'doctor', 'alternate': ['health_an...",0.77,[https://www.newportinstitute.com/?utm_source=...,None,None,"[""8445450602""]",{'names': {}},"[{'freeform': '43821 Shady Creek', 'locality':...",US,US
6,08f2aa879200ab960398d7c665470cf4,562949954974969,"[{'property': '', 'dataset': 'msft', 'record_i...",{'primary': 'Comfort Inn Alexandria West - Lan...,{'primary': 'hotel'},0.770000,[https://www.choicehotels.com/virginia/alexand...,None,None,"[""7036423422""]",...,{'primary': 'hotel'},0.77,[],None,None,"[""7036423422""]",{'names': {}},"[{'freeform': '6254 Duke St', 'locality': 'Ale...",US,US


In [359]:
yelp_df.map(type)

,business_id,name,address,city,state,postal_code,categories
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...
150341,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
150342,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
150343,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
150344,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [360]:
yelp_df.head()

,business_id,name,address,city,state,postal_code,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,"Department Stores, Shopping, Fashion, Home & G..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,"Brewpubs, Breweries, Food"


In [361]:
sample_df.head()

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses,country,base_country
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'shipping_center', 'alternate': ['...",0.996261,[http://www.goinpostaljacksonville.com/],[https://www.facebook.com/463273470392736],None,"[""+19049989600""]",...,"{'primary': 'vehicle_shipping', 'alternate': [...",0.77,[https://www.goinpostaljacksonville.com/],None,None,"[""9049989600""]",{'names': {}},"[{'freeform': '7643 Gate Pkwy Ste 104', 'local...",US,US
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[http://valleytransca.com/],[https://www.facebook.com/107150256087765],None,"[""+16194474353""]",...,"{'primary': 'transmission_repair', 'alternate'...",0.77,[http://valleytransca.com/],None,None,"[""6194474353""]",{'names': {}},"[{'freeform': '1158 N 2nd St', 'locality': 'El...",US,US
3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,[https://stores.redwingshoes.com/roswell-ga],None,None,"[""7705584400""]",...,"{'primary': 'shoe_store', 'alternate': ['onlin...",0.77,[http://stores.redwing.com/roswell-ga],None,None,"[""7705584400""]",{'names': {}},"[{'freeform': '875 Mansell Rd Ste B1', 'locali...",US,US
5,1125899912143279,1407374890190043,"[{'dataset': 'Microsoft', 'record_id': '112589...",{'primary': 'Newport Institute'},"{'primary': 'doctor', 'alternate': ['health_an...",0.770000,[http://www.newportinstitute.com/?utm_source=s...,None,None,"[""8445450602""]",...,"{'primary': 'doctor', 'alternate': ['health_an...",0.77,[https://www.newportinstitute.com/?utm_source=...,None,None,"[""8445450602""]",{'names': {}},"[{'freeform': '43821 Shady Creek', 'locality':...",US,US
6,08f2aa879200ab960398d7c665470cf4,562949954974969,"[{'property': '', 'dataset': 'msft', 'record_i...",{'primary': 'Comfort Inn Alexandria West - Lan...,{'primary': 'hotel'},0.770000,[https://www.choicehotels.com/virginia/alexand...,None,None,"[""7036423422""]",...,{'primary': 'hotel'},0.77,[],None,None,"[""7036423422""]",{'names': {}},"[{'freeform': '6254 Duke St', 'locality': 'Ale...",US,US


## Ground Truth Labeling: Comparing Sample Data with Yelp

In [362]:
# Extract and normalize address components from sample_df

def extract_address_component(addr_list, component):
    """Extract a specific component from address list"""
    if isinstance(addr_list, list) and addr_list:
        addr_dict = addr_list[0]
        if isinstance(addr_dict, dict):
            return addr_dict.get(component, '')
    return ''

def normalize_street_address(address):
    """Normalize street address for comparison"""
    if not address:
        return ''
    # Convert to lowercase and strip whitespace
    normalized = str(address).lower().strip()
    return normalized

def normalize_city(city):
    """Normalize city name for comparison"""
    if not city:
        return ''
    # Convert to lowercase and strip whitespace
    normalized = str(city).lower().strip()
    return normalized

def normalize_state(state):
    """Normalize state for comparison"""
    if not state:
        return ''
    # Convert to uppercase and strip whitespace
    normalized = str(state).upper().strip()
    return normalized

def normalize_postcode(postcode):
    """Normalize postcode - remove ZIP+4 extension"""
    if not postcode:
        return ''
    # Convert to string and strip whitespace
    normalized = str(postcode).strip()
    # Remove ZIP+4 extension (keep only first 5 digits)
    if '-' in normalized:
        normalized = normalized.split('-')[0]
    # Ensure it's exactly 5 digits if it looks like a ZIP code
    if normalized.isdigit() and len(normalized) == 5:
        return normalized
    elif normalized.isdigit() and len(normalized) < 5:
        # Pad with leading zeros if needed
        return normalized.zfill(5)
    else:
        return normalized

def extract_primary_name(name_data):
    """Extract primary name from name structure"""
    if isinstance(name_data, dict):
        primary = name_data.get('primary', '')
        if primary:
            return str(primary)
    elif isinstance(name_data, str):
        return str(name_data)
    return ''

def normalize_name(name):
    """Normalize business name for comparison"""
    
    if not name:
        return ''
    
    # Convert to string and lowercase
    normalized = str(name).lower().strip()
    
    # Remove common business suffixes and prefixes
    normalized = re.sub(r'\b(llc|inc|corp|corporation|co|ltd|limited|the)\b', '', normalized)
    
    # Remove special characters except spaces, hyphens, apostrophes, and ampersands
    normalized = re.sub(r"[^a-z0-9\s&\-']", ' ', normalized)
    
    # Normalize whitespace
    normalized = re.sub(r'\s+', ' ', normalized).strip()
    
    return normalized

print("Normalization functions created!")
print("Functions available:")
print("- extract_address_component()")
print("- normalize_street_address()")
print("- normalize_city()")
print("- normalize_state()")
print("- normalize_postcode()")
print("- extract_primary_name()")
print("- normalize_name()")

# Extract and normalize names
print("\nExtracting and normalizing names...")
sample_df['name_primary'] = sample_df['names'].apply(extract_primary_name)
sample_df['base_name_primary'] = sample_df['base_names'].apply(extract_primary_name)

sample_df['name_norm'] = sample_df['name_primary'].apply(normalize_name)
sample_df['base_name_norm'] = sample_df['base_name_primary'].apply(normalize_name)

# Extract address components for non-base addresses
print("Extracting address components from non-base addresses...")
sample_df['address_street'] = sample_df['addresses'].apply(lambda x: extract_address_component(x, 'freeform'))
sample_df['address_city'] = sample_df['addresses'].apply(lambda x: extract_address_component(x, 'locality'))
sample_df['address_state'] = sample_df['addresses'].apply(lambda x: extract_address_component(x, 'region'))
sample_df['address_postcode'] = sample_df['addresses'].apply(lambda x: extract_address_component(x, 'postcode'))

# Extract address components for base addresses
print("Extracting address components from base addresses...")
sample_df['base_address_street'] = sample_df['base_addresses'].apply(lambda x: extract_address_component(x, 'freeform'))
sample_df['base_address_city'] = sample_df['base_addresses'].apply(lambda x: extract_address_component(x, 'locality'))
sample_df['base_address_state'] = sample_df['base_addresses'].apply(lambda x: extract_address_component(x, 'region'))
sample_df['base_address_postcode'] = sample_df['base_addresses'].apply(lambda x: extract_address_component(x, 'postcode'))

# Normalize all address components
print("Normalizing address components...")

# Normalize non-base address components
sample_df['address_street_norm'] = sample_df['address_street'].apply(normalize_street_address)
sample_df['address_city_norm'] = sample_df['address_city'].apply(normalize_city)
sample_df['address_state_norm'] = sample_df['address_state'].apply(normalize_state)
sample_df['address_postcode_norm'] = sample_df['address_postcode'].apply(normalize_postcode)

# Normalize base address components
sample_df['base_address_street_norm'] = sample_df['base_address_street'].apply(normalize_street_address)
sample_df['base_address_city_norm'] = sample_df['base_address_city'].apply(normalize_city)
sample_df['base_address_state_norm'] = sample_df['base_address_state'].apply(normalize_state)
sample_df['base_address_postcode_norm'] = sample_df['base_address_postcode'].apply(normalize_postcode)

print("Name and address extraction and normalization complete!")

# Show examples of name normalization
print("\n=== NAME NORMALIZATION EXAMPLES ===")
for i in range(3):
    row = sample_df.iloc[i]
    print(f"\nExample {i+1}:")
    print(f"  Original names: {row['names']}")
    print(f"  Primary name: '{row['name_primary']}'")
    print(f"  Normalized name: '{row['name_norm']}'")
    print(f"  Original base_names: {row['base_names']}")
    print(f"  Primary base name: '{row['base_name_primary']}'")
    print(f"  Normalized base name: '{row['base_name_norm']}'")

# Display the key columns
name_and_address_columns = [
    'name_primary', 'base_name_primary', 'name_norm', 'base_name_norm',
    'address_street_norm', 'address_city_norm', 'address_state_norm', 'address_postcode_norm',
    'base_address_street_norm', 'base_address_city_norm', 'base_address_state_norm', 'base_address_postcode_norm'
]

print(f"\nFirst 5 rows of normalized names and addresses:")
sample_df[name_and_address_columns].head()

Normalization functions created!
Functions available:
- extract_address_component()
- normalize_street_address()
- normalize_city()
- normalize_state()
- normalize_postcode()
- extract_primary_name()
- normalize_name()

Extracting and normalizing names...
Extracting address components from non-base addresses...
Extracting address components from base addresses...
Normalizing address components...
Name and address extraction and normalization complete!

=== NAME NORMALIZATION EXAMPLES ===

Example 1:
  Original names: {'primary': "Goin' Postal Jacksonville"}
  Primary name: 'Goin' Postal Jacksonville'
  Normalized name: 'goin' postal jacksonville'
  Original base_names: {'primary': "Goin' Postal Jacksonville"}
  Primary base name: 'Goin' Postal Jacksonville'
  Normalized base name: 'goin' postal jacksonville'

Example 2:
  Original names: {'primary': 'Valley Transmission'}
  Primary name: 'Valley Transmission'
  Normalized name: 'valley transmission'
  Original base_names: {'primary': '

,name_primary,base_name_primary,name_norm,base_name_norm,address_street_norm,address_city_norm,address_state_norm,address_postcode_norm,base_address_street_norm,base_address_city_norm,base_address_state_norm,base_address_postcode_norm
0,Goin' Postal Jacksonville,Goin' Postal Jacksonville,goin' postal jacksonville,goin' postal jacksonville,7643 gate pkwy,jacksonville,FL,32256,7643 gate pkwy ste 104,jacksonville,FL,32256
1,Valley Transmission,Valley Transmission,valley transmission,valley transmission,1158 n 2nd st,el cajon,CA,92021,1158 n 2nd st,el cajon,CA,92021
3,"Red Wing - Roswell, GA",Red Wing,red wing - roswell ga,red wing,875 mansell rd b1,roswell,GA,30076,875 mansell rd ste b1,roswell,GA,30076
5,Newport Institute,Newport Institute,newport institute,newport institute,43821 shady crk,temecula,CA,92590,43821 shady creek,temecula,CA,92590
6,Comfort Inn Alexandria West - Landmark,Comfort Inn Landmark,comfort inn alexandria west - landmark,comfort inn landmark,6254 duke st,alexandria,VA,22312,6254 duke st,alexandria,VA,22312


In [363]:
# Normalize Yelp dataset address components for comparison
print("=== NORMALIZING YELP DATASET FOR COMPARISON ===")

# Normalize Yelp address components using the same functions
yelp_df['address_norm'] = yelp_df['address'].apply(normalize_street_address)
yelp_df['city_norm'] = yelp_df['city'].apply(normalize_city)
yelp_df['state_norm'] = yelp_df['state'].apply(normalize_state)
yelp_df['postal_code_norm'] = yelp_df['postal_code'].apply(normalize_postcode)

# Also normalize Yelp business names
yelp_df['name_norm'] = yelp_df['name'].apply(normalize_name)

print("Yelp dataset normalization complete!")
print(f"Yelp dataset size: {len(yelp_df)} businesses")

# Create consolidated address dictionaries for sample_df
print("\n=== CREATING CONSOLIDATED ADDRESS DICTIONARIES ===")

def create_address_dict(street, city, state, postcode):
    """Create a consolidated address dictionary from normalized components"""
    return {
        'address': street if street else '',
        'city': city if city else '',
        'state': state if state else '',
        'postal_code': postcode if postcode else ''
    }

# Create consolidated address dictionaries for sample_df
sample_df['address_norm'] = sample_df.apply(lambda row: create_address_dict(
    row['address_street_norm'],
    row['address_city_norm'], 
    row['address_state_norm'],
    row['address_postcode_norm']
), axis=1)

sample_df['base_address_norm'] = sample_df.apply(lambda row: create_address_dict(
    row['base_address_street_norm'],
    row['base_address_city_norm'],
    row['base_address_state_norm'], 
    row['base_address_postcode_norm']
), axis=1)

# Create consolidated address dictionary for Yelp data
yelp_df['address_dict_norm'] = yelp_df.apply(lambda row: create_address_dict(
    row['address_norm'],
    row['city_norm'],
    row['state_norm'],
    row['postal_code_norm']
), axis=1)




=== NORMALIZING YELP DATASET FOR COMPARISON ===
Yelp dataset normalization complete!
Yelp dataset size: 150346 businesses

=== CREATING CONSOLIDATED ADDRESS DICTIONARIES ===
Yelp dataset normalization complete!
Yelp dataset size: 150346 businesses

=== CREATING CONSOLIDATED ADDRESS DICTIONARIES ===


In [364]:
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,address_street_norm,address_city_norm,address_state_norm,address_postcode_norm,base_address_street_norm,base_address_city_norm,base_address_state_norm,base_address_postcode_norm,address_norm,base_address_norm
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'shipping_center', 'alternate': ['...",0.996261,[http://www.goinpostaljacksonville.com/],[https://www.facebook.com/463273470392736],None,"[""+19049989600""]",...,7643 gate pkwy,jacksonville,FL,32256,7643 gate pkwy ste 104,jacksonville,FL,32256,"{'address': '7643 gate pkwy', 'city': 'jackson...","{'address': '7643 gate pkwy ste 104', 'city': ..."
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[http://valleytransca.com/],[https://www.facebook.com/107150256087765],None,"[""+16194474353""]",...,1158 n 2nd st,el cajon,CA,92021,1158 n 2nd st,el cajon,CA,92021,"{'address': '1158 n 2nd st', 'city': 'el cajon...","{'address': '1158 n 2nd st', 'city': 'el cajon..."
3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,[https://stores.redwingshoes.com/roswell-ga],None,None,"[""7705584400""]",...,875 mansell rd b1,roswell,GA,30076,875 mansell rd ste b1,roswell,GA,30076,"{'address': '875 mansell rd b1', 'city': 'rosw...","{'address': '875 mansell rd ste b1', 'city': '..."
5,1125899912143279,1407374890190043,"[{'dataset': 'Microsoft', 'record_id': '112589...",{'primary': 'Newport Institute'},"{'primary': 'doctor', 'alternate': ['health_an...",0.770000,[http://www.newportinstitute.com/?utm_source=s...,None,None,"[""8445450602""]",...,43821 shady crk,temecula,CA,92590,43821 shady creek,temecula,CA,92590,"{'address': '43821 shady crk', 'city': 'temecu...","{'address': '43821 shady creek', 'city': 'teme..."
6,08f2aa879200ab960398d7c665470cf4,562949954974969,"[{'property': '', 'dataset': 'msft', 'record_i...",{'primary': 'Comfort Inn Alexandria West - Lan...,{'primary': 'hotel'},0.770000,[https://www.choicehotels.com/virginia/alexand...,None,None,"[""7036423422""]",...,6254 duke st,alexandria,VA,22312,6254 duke st,alexandria,VA,22312,"{'address': '6254 duke st', 'city': 'alexandri...","{'address': '6254 duke st', 'city': 'alexandri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,08f44c10dd244c09032cb0144487c85b,dba914f3e9e247fb64f813c7,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Edward Jones- Financial Advisor:H...,"{'primary': 'financial_advising', 'alternate':...",0.231548,[http://www.edwardjones.com/],[https://www.facebook.com/536502823203299],None,"[""+16783446501""]",...,2795 main st w,snellville,GA,30078,2795 main st w ste 24b,snellville,GA,30078,"{'address': '2795 main st w', 'city': 'snellvi...","{'address': '2795 main st w ste 24b', 'city': ..."
1979,08f28d542c8d230903943451a3bc73cf,4bc4c2c3abf49521d686c593,"[{'property': '', 'dataset': 'Microsoft', 'rec...",{'primary': 'Starbucks'},"{'primary': 'coffee_shop', 'alternate': ['rest...",0.770000,[https://www.starbucks.com/store-locator/store...,None,None,"[""2067552683""]",...,1400 6th ave,seattle,WA,98101,400 pine st,seattle,WA,98101,"{'address': '1400 6th ave', 'city': 'seattle',...","{'address': '400 pine st', 'city': 'seattle', ..."
1981,08f264626394cd4b030ac3f573c76061,4ca4f68af47ea1434edd5221,"[{'property': '', 'dataset': 'Microsoft', 'rec...",{'primary': 'Bank of America ATM'},"{'primary': 'online_shop', 'alternate': ['bank...",0.770000,[https://locators.bankofamerica.com/mo/richmon...,None,None,"[""8444018500""]",...,6646 clayton rd,richmond heights,MO,63117,6646 clayton rd,saint louis,MO,6

In [365]:
yelp_df

,business_id,name,address,city,state,postal_code,categories,address_norm,city_norm,state_norm,postal_code_norm,name_norm,address_dict_norm
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,"Doctors, Traditional Chinese Medicine, Naturop...","1616 chapala st, ste 2",santa barbara,CA,93101,abby rappoport lac cmq,"{'address': '1616 chapala st, ste 2', 'city': ..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma...",87 grasso plaza shopping center,affton,MO,63123,ups store,"{'address': '87 grasso plaza shopping center',..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,"Department Stores, Shopping, Fashion, Home & G...",5255 e broadway blvd,tucson,AZ,85711,target,"{'address': '5255 e broadway blvd', 'city': 't..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",935 race st,philadelphia,PA,19107,st honore pastries,"{'address': '935 race st', 'city': 'philadelph..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,"Brewpubs, Breweries, Food",101 walnut st,green lane,PA,18054,perkiomen valley brewery,"{'address': '101 walnut st', 'city': 'green la..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,"Nail Salons, Beauty & Spas",3388 gateway blvd,edmonton,AB,T6J 5H2,binh's nails,"{'address': '3388 gateway blvd', 'city': 'edmo..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,"Pets, Nurseries & Gardening, Pet Stores, Hobby...",2813 bransford ave,nashville,TN,37204,wild birds unlimited,"{'address': '2813 bransford ave', 'city': 'nas..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,"Shopping, Jewelry, Piercing, Toy Stores, Beaut...","6020 e 82nd st, ste 46",indianapolis,IN,46250,claire's boutique,"{'address': '6020 e 82nd st, ste 46', 'city': ..."
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,"Fitness/Exercise Equipment, Eyewear & Optician...",2472 troy rd,edwardsville,IL,62025,cyclery & fitness center,"{'address': '2472 troy rd', 'city': 'edwardsvi..."
